In [4]:
import os
import ants
import numpy as np
import pandas as pd
import tensorflow as tf

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [5]:
description = pd.read_csv('data/test_data.csv')
description.head()

,Image Data ID,Filename Str,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I101432,"""I101432.nii""",023_S_0388,MCI,M,72,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,4/17/2007,NiFTI,11/21/2022
1,I101541,"""I101541.nii""",027_S_1385,AD,F,71,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/26/2008,NiFTI,11/21/2022
2,I102204,"""I102204.nii""",023_S_0331,MCI,F,67,6,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/28/2008,NiFTI,11/21/2022
3,I102502,"""I102502.nii""",031_S_0830,MCI,M,72,5,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/24/2008,NiFTI,11/21/2022
4,I102735,"""I102735.nii""",027_S_0835,MCI,M,74,5,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,39511,NiFTI,11/21/2022


In [6]:
def preprocess_nii(filename, path):
    
    test_db = 'data/test'
    
    REG_DB_SUBFOLDERS = ['AD/', 'MCI/', 'CN/']
    REG_Folder = 'data/registered_data/'
    Output_Folder = 'data/output/'
    
    moving = "data/tpl-MNI305_T1w.nii.gz"
    head_mask = "data/tpl-MNI305_desc-head_mask.nii.gz"
    brain_mask = "data/tpl-MNI305_desc-brain_mask.nii.gz"
    
    image_ID = filename[:-4]
    row_index = description.index[description['Image Data ID'] == image_ID].tolist()[0]
    row = description.iloc[row_index]
    label = row['Group']
    
    registration_file = os.path.join(REG_Folder, label, image_ID)
    output_file = os.path.join(Output_Folder)
    
    fixed = f"{path}/{filename}"
    ! antsRegistrationSyNQuick.sh -d 3 -f {fixed} -m {moving} -o {registration_file}_ -p f -n 4 
    ! antsBrainExtraction.sh -d 3 -a {registration_file}_InverseWarped.nii.gz -e {head_mask} -m {brain_mask} -o {output_file}{label}_{image_ID}_ 
    

In [ ]:
test_db = 'data/test'

for nii in os.listdir(test_db):
    try:
        preprocess_nii(nii, test_db)
    except RuntimeError:
        print('Exception with', os.path.join(nii))

In [7]:
DB_TF_3D_PATH = 'data/output'

TFREC_3D_SS_TRAIN = 'train.3D.skull_stripped.tfrecords'
TFREC_3D_SS_TEST = 'test.3D.skull_stripped.tfrecords'
TFREC_3D_SS_VAL = 'validation.3D.skull_stripped.tfrecords'

train_tfrec = os.path.join(DB_TF_3D_PATH, TFREC_3D_SS_TRAIN)
test_tfrec = os.path.join(DB_TF_3D_PATH, TFREC_3D_SS_TEST)
val_tfrec = os.path.join(DB_TF_3D_PATH, TFREC_3D_SS_VAL)

SHUFFLE_BUFFER = 100

In [8]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))
  
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def create_tf_record(tf_rec_filename):
    LABELS = {'CN': 0, 'MCI': 1, 'AD': 2}
    writer = tf.io.TFRecordWriter(tf_rec_filename)
    output_folder = "data/output"
    for nii in os.listdir(output_folder):
        if nii.endswith("_BrainExtractionBrain.nii.gz"):
            img = ants.image_read(os.path.join(output_folder, nii)).numpy()
            label = LABELS[nii.split("_")[0]]
            image_ID = nii.split("_")[1]
            age_and_sex = description.loc[description['Image Data ID'] == image_ID, ['Age', 'Sex']].iloc[0]
            feature = {'label': _int64_feature(label),
               'subject_age': _int64_feature(age_and_sex[0]),
               'subject_sex': _bytes_feature(age_and_sex[1].encode('utf-8')),
               'image_id': _bytes_feature(image_ID.encode('utf-8')),
               'image': _float_feature(img.ravel())}
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
        else:
            pass
    writer.close()

In [ ]:
create_tf_record(train_tfrec)

In [9]:
from antspynet.architectures import create_resnet_model_3d


model = create_resnet_model_3d((172, 220, 156, 1), number_of_classification_labels=3)
model.summary()

2022-11-25 17:29:12.857899: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-25 17:29:12.893475: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-25 17:29:12.894469: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-25 17:29:12.895005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other ope

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 172, 220, 1  0           []                               
                                56, 1)]                                                           
                                                                                                  
 conv3d (Conv3D)                (None, 86, 110, 78,  22016       ['input_1[0][0]']                
                                 64)                                                              
                                                                                                  
 batch_normalization (BatchNorm  (None, 86, 110, 78,  256        ['conv3d[0][0]']                 
 alization)                      64)                                                          

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['acc'])

In [11]:
def _decode(serialized_example):
    IMG_SHAPE = (172, 220, 156, 1)
    feature = tf.io.parse_single_example(
        serialized_example,
        features = {
            'image': tf.io.FixedLenFeature(IMG_SHAPE, tf.float32),
            'label': tf.io.FixedLenFeature([], tf.int64),
            'subject_sex': tf.io.FixedLenFeature([], tf.string),
            'subject_age': tf.io.FixedLenFeature([], tf.int64),
            'image_id': tf.io.FixedLenFeature([], tf.string)
        }
    )
    
    return feature['image'], feature['label']

In [12]:
def dataset_parser(filepath, batch_size, n_classes=3):
    
    dataset = tf.data.TFRecordDataset(filepath).map(_decode)
    
    dataset = dataset.shuffle(SHUFFLE_BUFFER).repeat()
    dataset = dataset.batch(batch_size)  
    dataset = dataset.prefetch(1)
    
    iterator = dataset.__iter__()#make_one_shot_iterator()
    image, label = iterator.get_next()
    label = tf.one_hot(label, n_classes)
    
    return image, label

In [13]:
image_tensor, label_tensor = dataset_parser(train_tfrec, 8)

#val_tensor = dataset_parser(val_tfrec, 8) 

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(x=image_tensor, y=label_tensor, 
                    epochs=20,
                    callbacks=[es],
                    verbose=1)

Epoch 1/20


2022-11-25 17:29:41.414265: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsync failed to allocate 192082592 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 0/4294443008
2022-11-25 17:29:41.417705: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:293] Stats: Limit:                      1692309915
InUse:                      3589450261
MaxInUse:                   3589450261
NumAllocs:                        1076
MaxAllocSize:               1511178240
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2022-11-25 17:29:41.417756: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:56] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2022-11-25 17:29:41.418097: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:59] 

ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv3d/Conv3D' defined at (most recent call last):
    File "/home/keef07/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/keef07/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/keef07/.pyenv/versions/3.10.6/envs/DS1016/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/keef07/.pyenv/versions/3.10.6/envs/DS1016/lib/python3.10/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/keef07/.pyenv/versions/3.10.6/envs/DS1016/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/keef07/.pyenv/versions/3.10.6/envs/DS1016/lib/python3.10/site-packages/tornado/platform/async